# PVFleets: Empirical Degradation Rates - Geospatial analysis

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pvdeg
import dask.array as da
import dask.dataframe as dd
import xarray as xr
import os

In [ ]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)

# Single location example

In [ ]:


API_KEY = 'UCaEeYw0eAQEYDRq8S0LIQTFlwznp9AwBqTaLOXH'
# The example API key here is for demonstation and is rate-limited per IP.
# To get your own API key, visit https://developer.nrel.gov/signup/

weather_db = 'PSM3'
weather_id = (39.741931, -105.169891) # Golden, CO
weather_arg = {'api_key': API_KEY,
               'email': 'user@mail.com',
               'names': 'tmy',
               'attributes': [],
               'map_variables': True}

weather_df, meta = pvdeg.weather.get(weather_db, weather_id, **weather_arg)


k_h = pvdeg.degradation.hydrolysis_driven_rate(weather_df, meta)
print(k_h)

k_th = pvdeg.fatigue.thermomechanical_driven_rate(weather_df, meta)
print(k_th)




weather_db = 'NSRDB'
weather_id = (39.741931, -105.169891) # Golden, CO

weather_arg = {'satellite': 'Americas',
               'names': 2022,
               'NREL_HPC': True,
               'map_variables': True}

weather_df, meta = pvdeg.weather.get(weather_db, weather_id, **weather_arg)

k_p = pvdeg.degradation.light_driven_rate(weather_df, meta)
print(k_p)


## 1 Start distributed compute cluster - DASK

In [ ]:
hpc = {'manager': 'local',
        'n_workers': 20,
        'threads_per_worker': 4,
        'memory_limit': '12GB'}

pvdeg.geospatial.start_dask(hpc);

In [ ]:
# Get weather data
weather_db = 'NSRDB'

weather_arg = {'satellite': 'Americas',
               'names': 2022,
               'NREL_HPC': True,
               'attributes': ['air_temperature', 'wind_speed', 'dhi', 'ghi', 'dni', 'relative_humidity', 'clearsky_ghi']}

weather_ds, meta_df = pvdeg.weather.get(weather_db, geospatial=True, **weather_arg)

In [ ]:
meta_CO = meta_df[meta_df['state'] == 'Colorado']
meta_CO_sub, gids_CO_sub = pvdeg.utilities.gid_downsampling(meta_CO, 4)
weather_CO_sub = weather_ds.sel(gid=meta_CO_sub.index)

meta_US = meta_df[meta_df['country'] == 'United States']
meta_US_sub, gids_US_sub = pvdeg.utilities.gid_downsampling(meta_US, 1)
weather_US_sub = weather_ds.sel(gid=meta_US_sub.index)

In [ ]:
geo_rh_CO = {'func': pvdeg.degradation.hydrolysis_driven_rate,
              'weather_ds': weather_CO_sub,
              'meta_df': meta_CO_sub}

geo_tm_CO = {'func': pvdeg.fatigue.thermomechanical_driven_rate,
            'weather_ds': weather_CO_sub,
            'meta_df': meta_CO_sub}

geo_p_CO = {'func': pvdeg.degradation.light_driven_rate,
            'weather_ds': weather_CO_sub,
            'meta_df': meta_CO_sub}

geo_rh_US = {'func': pvdeg.degradation.hydrolysis_driven_rate,
              'weather_ds': weather_US_sub,
              'meta_df': meta_US_sub}

geo_tm_US = {'func': pvdeg.fatigue.thermomechanical_driven_rate,
            'weather_ds': weather_US_sub,
            'meta_df': meta_US_sub}

geo_p_US = {'func': pvdeg.degradation.light_driven_rate,
            'weather_ds': weather_US_sub,
            'meta_df': meta_US_sub}

# hydrolysis_res_CO = pvdeg.geospatial.analysis(**geo_rh_CO)
# thermomechanical_res_CO = pvdeg.geospatial.analysis(**geo_tm_CO)
# photo_res_CO = pvdeg.geospatial.analysis(**geo_p_CO)
# hydrolysis_res = hydrolysis_res_CO
# thermomechanical_res = thermomechanical_res_CO
# photo_res = photo_res_CO

# hydrolysis_res_US = pvdeg.geospatial.analysis(**geo_rh_US)
# thermomechanical_res_US = pvdeg.geospatial.analysis(**geo_tm_US)
photo_res_US = pvdeg.geospatial.analysis(**geo_p_US)
# hydrolysis_res = hydrolysis_res_US
# thermomechanical_res = thermomechanical_res_US
photo_res = photo_res_US

In [ ]:
import os
work_dir = '/projects/pvsoiling/pvdeg/analysis/pvfleets'

# hydrolysis_res.to_netcdf(os.path.join(work_dir, "hydrolysis_res.nc"))
# thermomechanical_res.to_netcdf(os.path.join(work_dir, "thermomechanical_res.nc"))
photo_res.to_netcdf(os.path.join(work_dir, "photo_res.nc"))

# hydrolysis_res.to_dataframe().to_csv(os.path.join(work_dir, "hydrolysis_res.csv"))
# thermomechanical_res.to_dataframe().to_csv(os.path.join(work_dir, "thermomechanical_res.csv"))
photo_res.to_dataframe().to_csv(os.path.join(work_dir, "photo_res.csv"))


In [ ]:
fig, ax = pvdeg.geospatial.plot_USA(hydrolysis_res['k_h'],
                   cmap='viridis', vmin=0, vmax=0.2,
                   title='Hydrolysis driven degradation rate (%/year)',
                   cb_title='degradation rate (%/y)')

fig.savefig(os.path.join(work_dir, "hydrolysis_map.png"))

In [ ]:
fig, ax = pvdeg.geospatial.plot_USA(thermomechanical_res['k_th'],
                   cmap='viridis', vmin=0, vmax=None,
                   title='Thermo-mechanical driven degradation rate (%/year)',
                   cb_title='degradation rate (%/y)')

fig.savefig(os.path.join(work_dir, "thermomechanical_map.png"))

In [ ]:
fig, ax = pvdeg.geospatial.plot_USA(photo_res['k_p'],
                   cmap='viridis', vmin=0, vmax=None,
                   title='Light driven degradation rate (%/year)',
                   cb_title='degradation rate (%/y)')

fig.savefig(os.path.join(work_dir, "photo_deg_map.png"))

In [ ]:
pvfleets_loc = pd.read_csv('/projects/pvsoiling/pvdeg/analysis/pvfleets/pvdrdb_metadata.csv')
uspvdb_loc = pd.read_csv('/projects/pvsoiling/pvdeg/analysis/pvfleets/uspvdb_v1_0_20231108.csv')

hydrolysis_res = pd.read_csv('/projects/pvsoiling/pvdeg/analysis/pvfleets/hydrolysis_res.csv')
thermomechanical_res = pd.read_csv('/projects/pvsoiling/pvdeg/analysis/pvfleets/thermomechanical_res.csv')
photo_res = pd.read_csv('/projects/pvsoiling/pvdeg/analysis/pvfleets/photo_res.csv')

In [ ]:
spatial_degradation = hydrolysis_res.copy()
spatial_degradation['k_tm'] = thermomechanical_res['k_th']
spatial_degradation['k_p'] = photo_res['k_p']
spatial_degradation['k_T'] = 1 * (1 + spatial_degradation['k_h']) * (1+ spatial_degradation['k_tm']) * (1 + spatial_degradation['k_p']) - 1

In [ ]:
spatial_degradation['pvfleets_system_id'] = np.nan
spatial_degradation['uspvdb_system_id'] = np.nan
spatial_degradation['pvcz_labeled'] = ''
spatial_degradation['KG_zone'] = ''

In [ ]:
import pvcz
df_pvcz = pvcz.get_pvcz_data()

In [ ]:
for index, row in pvfleets_loc.iterrows():
    lat_poi = row['latitude']
    lon_poi = row['longitude']

    closest_index_map = pvcz.arg_closest_point(lat_poi, lon_poi, spatial_degradation['latitude'], spatial_degradation['longitude'])

    column = 'pvfleets_system_id'
    col = spatial_degradation.columns.get_loc(column)
    spatial_degradation.iloc[closest_index_map, col] = row['system_id']

In [ ]:
for index, row in uspvdb_loc.iterrows():
    lat_poi = row['ylat']
    lon_poi = row['xlong']

    closest_index_map = pvcz.arg_closest_point(lat_poi, lon_poi, spatial_degradation['latitude'], spatial_degradation['longitude'])

    column = 'uspvdb_system_id'
    col = spatial_degradation.columns.get_loc(column)
    spatial_degradation.iloc[closest_index_map, col] = row['OID_']

In [ ]:
for index, row in spatial_degradation[~spatial_degradation['k_T'].isnull()].iterrows():
    lat_poi = row['latitude']
    lon_poi = row['longitude']

    closest_index_pvcz = pvcz.arg_closest_point(lat_poi, lon_poi, df_pvcz['lat'], df_pvcz['lon'])

    col_pvcz = spatial_degradation.columns.get_loc('pvcz_labeled')
    col_KG = spatial_degradation.columns.get_loc('KG_zone')
    spatial_degradation.iloc[index, col_pvcz] = df_pvcz.iloc[closest_index_pvcz]['pvcz_labeled']
    spatial_degradation.iloc[index, col_KG] = df_pvcz.iloc[closest_index_pvcz]['KG_zone']

spatial_degradation.to_csv(os.path.join(work_dir, "geospatial_deg.csv"))

In [ ]:
spatial_degradation.to_csv(os.path.join(work_dir, "geospatial_deg.csv"))

In [ ]:
stacked = xr.Dataset.from_dataframe(spatial_degradation)
stacked = stacked.drop(["index"])
mindex_obj = pd.MultiIndex.from_arrays(
    [spatial_degradation["latitude"], spatial_degradation["longitude"]], names=["latitude", "longitude"]
)
mindex_coords = xr.Coordinates.from_pandas_multiindex(mindex_obj, "index")
stacked = stacked.assign_coords(mindex_coords)

stacked = stacked.drop_duplicates("index")
res = stacked.unstack("index")  # , sparse=True

In [ ]:
fig, ax = pvdeg.geospatial.plot_USA(res['k_T'],
                   cmap='viridis', vmin=0, vmax=None,
                   title='Total degradation rate (%/year)',
                   cb_title='degradation rate (%/y)')

fig.savefig(os.path.join(work_dir, "total_deg_map.png"))

In [ ]:
fig, ax = pvdeg.geospatial.plot_USA(res['k_T'],
                   cmap='viridis', vmin=0, vmax=None,
                   title='Total degradation rate (%/year)',
                   cb_title='degradation rate (%/y)')

fig.savefig(os.path.join(work_dir, "total_deg_map.png"))